In [0]:
#Librerias necesarias
!curl https://colab.chainer.org/install | sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   4293      0 --:--:-- --:--:-- --:--:--  4293
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.
+ pip install -q cupy-cuda100  chainer 
+ set +ex
Installation succeeded!


# Código revisado

In [0]:
import numpy as np
import cupy as cp
import solver.extraer_datos_yahoo as extrae
import solver.funciones_auxiliares as aux
import solver.line_search as line
import solver.modelo_markowitz as mkv
import solver.utils as utils
import solver.optimizacion_numerica as opt
#from utils import inc_index, dec_index, compute_error, norm_residual, condicion_cupy
#from line_search import line_search_by_backtracking, line_search_for_residual_by_backtracking

## Código módulos

### Extraer datos


In [0]:
import fix_yahoo_finance as yf

def extraer_datos_yahoo(stocks, start='2015-01-01', end='2020-04-30'):
  '''
  Funcion para extraer precios al cierre de las acciones mediante yahoo finance de 2015-01-01 a 2020-04-30

  params: stocks    lista de acciones de las cuales se desea obtener el precio
          start     fecha inicial
          end       fecha final

  return: base      Dataframe de precios por acción (columnas) y día (filas)
  '''
  df_c = yf.download(stocks, start=start, end=end).Close
  base = df_c['AAPL'].dropna().to_frame()
  for i in range(0,50):
      base = base.join(df_c.iloc[:,i].to_frame(), lsuffix='_caller', rsuffix='_other')
  base = base.drop(columns=['AAPL_caller'])
  base = base.rename(columns={"AAPL_other": "AAPL"})
  base = base.fillna(method='ffill')
  base = base.fillna(method='bfill')
  return base


### Funciones auxiliares

In [0]:
import cupy as cp

def calcular_rendimiento_vector(x):
  """
  Función para calcular el rendimiento esperado

  params:
      x     vector de precios
  
  return:
      r_est rendimiento esperado diario
  """

  # Definimos precios iniciales y finales como arreglo alojado en la gpu
  x_o = cp.asarray(x)
  x_f = x_o[1:]

  # Calculamos los rendimientos diarios
  r = cp.log(x_f/x_o[:-1])

  return r

def calcular_rendimiento(X):
  """
  Función para calcular el rendimiento esperado para un conjunto de acciones

  params:
      X      matriz mxn de precios, donde:
             m es el número de observaciones y
             n el número de acciones
  
  return:
      r_est rvector de rendimientos esperados
  """
  m,n = X.shape
  r_est = cp.zeros(n)
  X = cp.asarray(X)

  for i in range(n):
    r_est[i] = calcular_rendimiento_vector(X[:,i]).mean()

  return 264*r_est

def calcular_varianza(X):

  """
  Función para calcular el la matriz de varianzas y covarianzas para un conjunto de acciones

  params:
      X      matriz mxn de precios, donde:
             m es el número de observaciones y
               n el número de acciones
  
  return:
      S  matriz de varianzas y covarianzas
  """
  m,n=X.shape
  X = cp.asarray(X)

  X_m = cp.zeros((m-1,n))

  for i in range(n):
    X_m[:,i] = calcular_rendimiento_vector(X[:,i]) - calcular_rendimiento_vector(X[:,i]).mean()

  S = (cp.transpose(X_m)@X_m)/(m-2)

  return S




### Modelo Markovitz

In [0]:
import cupy as cp
import pandas as pd

def formar_vectores(mu, Sigma):
  '''
  Calcula las cantidades u = \Sigma^{-1}  \mu y v := \Sigma^{-1} \cdot 1 del problema de Markowitz

  Args:
    mu (cupy array, vector): valores medios esperados de activos (dimension n)
    Sigma (cupy array, matriz): matriz de covarianzas asociada a activos (dimension n x n)

  Return:
    u (cupy array, escalar): vector dado por \cdot Sigma^-1 \cdot mu (dimension n)
    v (cupy array, escalar): vector dado por Sigma^-1 \cdot 1 (dimension n)
  '''

  # Vector auxiliar con entradas igual a 1
  n = Sigma.shape[0]
  ones_vector = cp.ones(n)

  # Formamos vector \cdot Sigma^-1 mu y Sigm^-1 1
  # Nota: 
  #   1) u= Sigma^-1 \cdot mu se obtiene resolviendo  Sigma u = mu
  #   2) v= Sigma^-1 \cdot 1 se obtiene resolviendo  Sigma v = 1

  # Obtiene vectores de interes
  u = cp.linalg.solve(Sigma, mu)
  u = u.transpose() # correcion de expresion de array
  v = cp.linalg.solve(Sigma, ones_vector)

  return u , v

def formar_abc(mu, Sigma):
  '''
  Calcula las cantidades A, B y C del diagrama de flujo del problema de Markowitz

  Args:
    mu (cupy array, vector): valores medios esperados de activos (dimension n)
    Sigma (cupy array, matriz): matriz de covarianzas asociada a activos (dimension n x n)

  Return:
    A (cupy array, escalar): escalar dado por mu^t \cdot Sigma^-1 \cdot mu
    B (cupy array, escalar): escalar dado por 1^t \cdot Sigma^-1 \cdot 1
    C (cupy array, escalar): escalar dado por 1^t \cdot Sigma^-1 \cdot mu
  '''

  # Vector auxiliar con entradas igual a 1
  n = Sigma.shape[0]
  ones_vector = cp.ones(n)

  # Formamos vector \cdot Sigma^-1 mu y Sigm^-1 1
  # Nota: 
  #   1) u= Sigma^-1 \cdot mu se obtiene resolviendo  Sigma u = mu
  #   2) v= Sigma^-1 \cdot 1 se obtiene resolviendo  Sigma v = 1

  u, v = formar_vectores(mu, Sigma)

  # Obtiene escalares de interes
  A = mu.transpose()@u
  B = ones_vector.transpose()@v
  C = ones_vector.transpose()@u

  return A, B, C


def delta(A,B,C):
  '''
  Calcula las cantidad Delta = AB-C^2 del diagrama de flujo del problema de Markowitz

  Args:
    A (cupy array, escalar): escalar dado por mu^t \cdot Sigma^-1 \cdot mu
    B (cupy array, escalar): escalar dado por 1^t \cdot Sigma^-1 \cdot 1
    C (cupy array, escalar): escalar dado por 1^t \cdot Sigma^-1 \cdot mu

  Return:
    Delta (cupy array, escalar): escalar dado \mu^t \cdot \Sigma^{-1} \cdot \mu
  '''
  Delta = A*B-C**2

  return Delta


def formar_omegas(r, mu, Sigma):
  '''
  Calcula las cantidades w_o y w_ del problema de Markowitz

  Args:
    r (escalar) : retorno esperado por el inversionista
    mu (cupy array, vector): valores medios esperados de activos (dimension n)
    Sigma (cupy array, matriz): matriz de covarianzas asociada a activos (dimension n x n)

  Return:
    w_0 (cupy escalar, matriz): escalar dado por 
          w_0 = \frac{1}{\Delta} (B \Sigma^{-1} \hat{\mu}- C\Sigma^{-1} 1) 
    w_1 (cupy escalar, vector): escalar dado por 
         w_1 = \frac{1}{\Delta} (C \Sigma^{-1} \hat{\mu}- A\Sigma^{-1} 1)
  '''
  # Obtenemos u = Sigma^{-1} \hat{\mu}, v = \Sigma^{-1} 1
  u, v = formar_vectores(mu, Sigma)
  # Escalares relevantes
  A, B, C = formar_abc(mu, Sigma)
  Delta = delta(A,B,C)
  # Formamos w_0 y w_1
  w_0 = (1/Delta)*(r*B-C)
  w_1 = (1/Delta)*(A-C*r)

  return w_0, w_1

def markowitz(r, mu, Sigma):
  '''
  Calcula las cantidades w_o y w_ del problema de Markowitz

  Args:
    r (escalar) : retorno esperado por el inversionista
    mu (cupy array, vector): valores medios esperados de activos (dimension n)
    Sigma (cupy array, matriz): matriz de covarianzas asociada a activos (dimension n x n)

  Return:
    w (cupy array, vector): vector de pesos que minimizan la varianza del portfolio
  '''
  # Obtenemos u = Sigma^{-1} \hat{\mu}, v = \Sigma^{-1} 1
  u, v = formar_vectores(mu, Sigma)

  # Formamos w_0 y w_1
  w_0, w_1 = formar_omegas(r, mu, Sigma)

  return w_0*u+w_1*v

def markowitz_df(r, mu, Sigma, stocks):
  '''
  Crea data frame de la función markowitz relacionando los pesos con las acciones

  Args:
    r (escalar) : retorno esperado por el inversionista
    mu (cupy array, vector): valores medios esperados de activos (dimension n)
    Sigma (cupy array, matriz): matriz de covarianzas asociada a activos (dimension n x n)
    stocks (lista): Códigos bursátiles de las acciones a evaluar

  Return:
    df (DataFrame): Data Frame con acciones como índices y los pesos ordenados correspondientes
  '''

  aux = markowitz(r, mu, Sigma)
  df =pd.DataFrame( cp.asnumpy(aux), columns = ['pesos'])
  df["stocks"]=stocks
  df = df.sort_values(by=['pesos'], ascending=False)
  return df




### Optimización numérica

In [0]:
import cupy as cp
import numpy as np
from solver.utils import inc_index
from solver.utils import dec_index


def gfo_cp_mark(Sigma,x):
    '''
    gradiente de la función objetivo 1/2*x.t*Sigma*x
    input: matriz Sigma y vector x
    output: producto matriz-vector Sigma*x
    '''
    first_block = Sigma@x
    return first_block


def Hfo_cp_mark(Sigma):
    '''
    Hessiana de la función objetivo 1/2*x.t*Sigma*x
    input: matriz Sigma
    output: matriz Sigma
    '''
    first_block = Sigma 
    return first_block


def gradient_approximation(f,x,h=1e-8):
    '''
    Numerical approximation of gradient for function f using forward differences.
    Args:
        f (lambda expression): definition of function f.
        x (array): numpy array that holds values where gradient will be computed.
        h (float): step size for forward differences, tipically h=1e-8
    Returns:
        gf (array): numerical approximation to gradient of f.
    '''
    n = x.size
    gf = cp.zeros(n)
    f_x = f(x)
    for i in np.arange(n):
        inc_index(x,i,h)
        gf[i] = f(x) - f_x
        dec_index(x,i,h)
    return gf/h



def Hessian_approximation(f,x,h=1e-6):
    '''
    Numerical approximation of Hessian for function f using forward differences.
    Args:
        f (lambda expression): definition of function f.
        x (array): numpy array that holds values where Hessian will be computed.
        h (float): step size for forward differences, tipically h=1e-6
    Returns:
        Hf (array): numerical approximation to Hessian of f.
    '''
    n = x.size
    Hf = cp.zeros((n,n))
    f_x = f(x)
    for i in np.arange(n):
        inc_index(x,i,h)
        f_x_inc_in_i = f(x)
        for j in np.arange(i,n):
            inc_index(x,j,h)
            f_x_inc_in_i_j = f(x)
            dec_index(x,i,h)
            f_x_inc_in_j = f(x)
            dif = f_x_inc_in_i_j-f_x_inc_in_i-f_x_inc_in_j+f_x
            Hf[i,j] = dif
            if j != i:
                Hf[j,i] = dif
            dec_index(x,j,h)
            inc_index(x,i,h)
        dec_index(x,i,h)
    return Hf/h**2

import solver



def Newtons_method_feasible_init_point(f, A, x_0, tol, 
                                       tol_backtracking, x_ast=None, p_ast=None, maxiter=30,
                                       gf_symbolic = None,
                                       Hf_symbolic = None,
                                       Sigma = None):
    '''
    Newton's method to numerically approximate solution of min f subject to Ax = b.
    IMPORTANT: this implementation requires that initial point x_0, satisfies: Ax_0 = b
    Args:
        f (fun): definition of function f as lambda expression or function definition.
        A (numpy ndarray): 2d numpy array of shape (m,n) defines system of constraints Ax=b.
        x_0 (numpy ndarray): initial point for Newton's method. Must satisfy: Ax_0 = b
        tol (float): tolerance that will halt method. Controls stopping criteria.
        tol_backtracking (float): tolerance that will halt method. Controls value of line search by backtracking.
        x_ast (numpy ndarray): solution of min f, now it's required that user knows the solution...
        p_ast (float): value of f(x_ast), now it's required that user knows the solution...
        maxiter (int): maximum number of iterations
        gf_symbolic (fun): definition of gradient of f. If given, no approximation is
                                     performed via finite differences.
        Hf_symbolic (fun): definition of Hessian of f. If given, no approximation is
                                     performed via fi
                                     nite differences.
    Returns:
        x (numpy ndarray): numpy array, approximation of x_ast.
        iteration (int): number of iterations.
        Err_plot (numpy ndarray): numpy array of absolute error between p_ast and f(x) with x approximation
                          of x_ast. Useful for plotting.
        x_plot (numpy ndarray): numpy array that containts in columns vector of approximations. Last column
                        contains x, approximation of solution. Useful for plotting.
    '''
    iteration = 0
        
    x = x_0
    
    feval = f(x)
    
    if gf_symbolic:
        gfeval = gf_symbolic(x,Sigma)
    else:
        gfeval = gradient_approximation(f,x)

    if Hf_symbolic:
        Hfeval = Hf_symbolic(x,Sigma)
    else:
        Hfeval = Hessian_approximation(f,x)
    
    normgf = np.linalg.norm(gfeval)
    condHf= solver.utils.condicion_cupy(Hfeval)
    
    Err_plot_aux = np.zeros(maxiter)
    Err_plot_aux[iteration]= solver.utils.compute_error(p_ast,feval)
    
    Err = solver.utils.compute_error(x_ast,x)
    
        
    if(A.ndim == 1):
        p = 1
        n = x.size
        zero_matrix = cp.zeros(p)
        first_stack = cp.column_stack((Hfeval, A.T))
        second_stack = cp.row_stack((A.reshape(1,n).T,zero_matrix)).reshape(1,n+1)[0]
    else:
        p,n = A.shape
        zero_matrix = cp.zeros((p,p))
        first_stack = np.column_stack((Hfeval, A.T))
        second_stack = np.column_stack((A,zero_matrix))
        
    x_plot = cp.zeros((n,maxiter))
    x_plot[:,iteration] = x
    
    system_matrix = cp.vstack((first_stack,second_stack))
    zero_vector = cp.zeros(p)
    rhs = cp.vstack((gfeval.reshape(n,1), zero_vector.reshape(p,1))).T[0]

    #Newton's direction and Newton's decrement
    dir_desc = cp.linalg.solve(system_matrix, -rhs)
    dir_Newton = dir_desc[0:n]
    dec_Newton = -gfeval.dot(dir_Newton)
    w_dual_variable_estimation = dir_desc[n:(n+p)]


    print('I\tNormgf \tNewton Decrement\tError x_ast\tError p_ast\tline search\tCondHf')
    print('{}\t{}\t{}\t{}\t{}\t{}\t\t{}'.format(iteration,
                                                cp.around(normgf,4),
                                                cp.around(dec_Newton,4),
                                                cp.around(Err,4),
                                                cp.around(Err_plot_aux[iteration],4),
                                                "---",
                                                cp.around(condHf,4)))
    stopping_criteria = dec_Newton/2
    iteration+=1
    while(stopping_criteria>tol and iteration < maxiter):
        der_direct = -dec_Newton
        t = solver.line_search.line_search_by_backtracking(f,dir_Newton,x,der_direct)
        x = x + t*dir_Newton
        feval = f(x)
        
        
        if gf_symbolic:
            gfeval = gf_symbolic(x,Sigma)
        else:
            gfeval = gradient_approximation(f,x)
        
        if Hf_symbolic: 
            Hfeval = Hf_symbolic(x,Sigma)
        else:
            Hfeval = Hessian_approximation(f,x)
        if(A.ndim == 1):
            first_stack = cp.column_stack((Hfeval, A.T))
        else:
            first_stack = cp.column_stack((Hfeval, A.T))

        system_matrix = cp.vstack((first_stack,second_stack))
        rhs = cp.vstack((gfeval.reshape(n,1), zero_vector.reshape(p,1))).T[0]
        #Newton's direction and Newton's decrement
        dir_desc = cp.linalg.solve(system_matrix, -rhs)
        dir_Newton = dir_desc[0:n]
        dec_Newton = -gfeval.dot(dir_Newton)
        w_dual_variable_estimation = dir_desc[n:(n+p)]
        
        Err_plot_aux[iteration]= solver.utils.compute_error(p_ast,feval)
        x_plot[:,iteration] = x
        Err = solver.utils.compute_error(x_ast,x)
        print('{}\t{}\t{}\t{}\t{}\t{}\t{}'.format(iteration,
                                                  cp.around(normgf,4),
                                                  cp.around(dec_Newton,4),
                                                  cp.around(Err,4),
                                                  cp.around(Err_plot_aux[iteration],4),
                                                  cp.around(t,4),
                                                  cp.around(condHf,4)))
        stopping_criteria = dec_Newton/2
        if t<tol_backtracking: #if t is less than tol_backtracking then we need to check the reason
            iter_salida=iteration
            iteration = maxiter - 1
        iteration+=1
    print('{} {}'.format("Error of x with respect to x_ast:",Err))
    print('{} {}'.format("Approximate solution:", x))
    cond = Err_plot_aux > np.finfo(float).eps*10**(-2)
    Err_plot = Err_plot_aux[cond]
    
    if iteration == maxiter and t < tol_backtracking:
        print("Backtracking value less than tol_backtracking, check approximation")
        iteration=iter_salida
    else:
        if iteration == maxiter:
            print("Reached maximum of iterations, check approximation")
    x_plot = x_plot[:,:iteration]
    return [x,iteration,Err_plot,x_plot]


### Utils

In [0]:
import cupy as cp

def inc_index(vec,index,h):
    '''
    Funcion auxiliar para calculo gradiente y hessiano.
    Args:
        vec (array): array de cupy.
        index (int): indice.
        h (float):   cantidad del vec[index] que sera incrementada.
    Returns:
        vec (array): array vec de cupy con vec[index] incrementado con h.
    '''
    vec[index] +=h
    return vec

def dec_index(vec,index,h=1):
    '''
    Funcion auxiliar para calculo gradiente y hessiano.
    Args:
        vec (array): array de cupy.
        index (int): indice.
        h (float):   antidad del vec[index] que sera disminuida.
    Returns:
        vec (array): array vec de cupy con vec[index] disminuida con h.
    '''
    vec[index] -=h
    return vec

def compute_error(x_obj,x_approx):
    '''
    Error absoluto o relativo entre x_obj y x_approx.
    '''
    if cp.linalg.norm(x_obj) > cp.nextafter(0,1):
        Err=cp.linalg.norm(x_obj-x_approx)/cp.linalg.norm(x_obj)
    else:
        Err=cp.linalg.norm(x_obj-x_approx)
    return Err

def norm_residual(feas_primal, feas_dual):
    '''
    Calcula la norma de residuos para el método de punto inicial no factible de Newton
    '''
    return cp.sqrt(cp.linalg.norm(feas_primal)**2 +\
                   cp.linalg.norm(feas_dual)**2
                   )

def condicion_cupy(A):
    '''
    Función que calcula la condición de una matriz para matrices en Cupy
    Args:
      A:  Matriz de cupy
    Return:
      c: Condición de una matriz
    '''
    A_inv = cp.linalg.inv(A)
    NAi = cp.linalg.norm(A_inv)
    NA = cp.linalg.norm(A)
    return NAi*NA

def feasible_markowitz(r,mu):
  '''
  Funcion que devuelve un punto factible del problema de Markowitz, resolviendo
  las ecuaciones normales asociadas
  Args:
     r (float): real positivo, que modela rendimiento esperado por inversionista
     mu (aray): array de cupy que contiene los valores medios de los rendimientos
          de los activos en el portafolio
  Return:
     w_star (float): punto feasible del problema de Markowitz
  '''
  # matriz del sistema para ecuaciones normales
  M = cp.ones((2,mu.shape[0]))

  for i in range(mu.shape[0]):
    M[0,i]=mu[i]

  # lado derecho de sistema
  b= cp.array([r,1])

  # Resuelve ecuaciones normales
  w_star = cp.linalg.solve(M.transpose()@M,M.transpose()@b)
  
  return w_star

### Punto factible

In [0]:
import cupy as cp
def feasible_markowitz(r,mu):
  '''
  Funcion que devuelve un punto factible del problema de Markowitz, resolviendo
  las ecuaciones normales asociadas
  Args:
     r (float): real positivo, que modela rendimiento esperado por inversionista
     mu (aray): array de cupy que contiene los valores medios de los rendimientos
          de los activos en el portafolio
  Return:
     w_star (float): punto feasible del problema de Markowitz
  '''
  # matriz del sistema para ecuaciones normales
  M = cp.ones((2,mu.shape[0]))

  for i in range(mu.shape[0]):
    M[0,i]=mu[i,0]

  # lado derecho de sistema
  b= cp.array([r,1])

  # Resuelve ecuaciones normales
  w_star = cp.linalg.solve(M.transpose()@M,M.transpose()@b)
  
  return w_star

### Line search

In [0]:
from solver.utils import norm_residual

def line_search_by_backtracking(f,dir_desc,x,
                                der_direct, alpha=.15, beta=.5):
    '''
    Busqueda de linea que disminuye suficientemente f restringida a un rayo en
    la dirección dir_desc.
    Args:
        alpha (float): parametro en busqueda de linea con backtracking, tipicamente .15
        beta (float): parametro en busqueda de linea con backtracking, tipicamente .5
        f (lambda expression): definicion de funcion f.
        dir_desc (array): direccion de descenso.
        x (array): array de cupy que contiene valores donde se realizara la busqueda de linea
        der_direct (float): derivada direccional de f.
    Returns:
        t (float): numero positivo para el tamano de pasos a lo largo de dir_desc que
                   disminuye suficientemente f.
    '''
    t=1
    if alpha > 1/2:
        print('alpha menor o igual que 1/2')
        t=-1
    if beta>1:
        print('beta debe ser menor a 1')
        t=-1;
    if t!=-1:
        eval1 = f(x+t*dir_desc)
        eval2 = f(x) + alpha*t*der_direct
        while eval1 > eval2:
            t=beta*t
            eval1=f(x+t*dir_desc)
            eval2=f(x)+alpha*t*der_direct
    else:
        t=-1
    return t

def line_search_for_residual_by_backtracking(r_primal, r_dual,dir_desc_primal,dir_desc_dual,x, nu,
                                             norm_residual_eval,
                                             alpha=.15, beta=.5):
    '''
    Busqueda de linea que disminuye suficientemente el residual para el metodo
    de punto inicial inviable de Newton restringido a un rayo en la direccion dir_desc.
    Args:
        r_primal (fun): definicion de residual primal como funcion definida o expresion lambda.
        r_dual (fun): definicion de residual dual como funcion definida o expresion lambda.
        dir_desc_primal (array): direccion de desceso para variable primal.
        dir_desc_dual (array): direccion de descenso para variable dual.
        x (array): array de cupy que contiene valores donde se realizara la busqueda de linea.
        nu (array): array de cupy que contiene valores donde se realizara la busqueda de linea.
        norm_residual_eval (float): norma de residuos que tiene evaluaciones r_primal y 
        r_dual en x y nu
        alpha (float): parametro de busqueda de linea con backtracking, tipicamente .15
        beta (float): parametro de busqueda de linea con backtracking, tipicamente .5

    Returns:
        t (float): numero positivo para el tamano de pasos a lo largo de dir_desc que 
        disminuye suficientemente f.
    '''
    t=1
    if alpha > 1/2:
        print('alpha debe ser menor o igual que 1/2')
        t=-1
    if beta>1:
        print('beta debe ser menor que uno 1')
        t=-1;
    if t!=-1:
        feas_primal = r_primal(x + t*dir_desc_primal)
        feas_dual = r_dual(nu + t*dir_desc_dual )
        eval1 = norm_residual(feas_primal, feas_dual)
        eval2 = (1-alpha*t)*norm_residual_eval
        while eval1 > eval2:
            t=beta*t
            feas_primal = r_primal(x + t*dir_desc_primal)
            feas_dual = r_dual(nu + t*dir_desc_dual )
            eval1 = norm_residual(feas_primal, feas_dual)
            eval2 = (1-alpha*t)*norm_residual_eval
    return t


## Cargamos la información y calculamos los parámetros

Primero cargamos los precios de las acciones que se han escogido para esta implementación

In [0]:
stocks = ['COP','AMT','LIN','LMT','AMZN','WMT','JNJ','VTI','MSFT','GOOG','XOM','CCI','BHP.AX','UNP',
'BABA','NSRGY','RHHBY','VOO','AAPL','FB','CVX','PLD','RIO.L','HON','HD','PG','UNH','BRK-A','V','0700.HK',
'RDSA.AS','0688.HK','AI.PA','RTX','MC.PA','KO','PFE','JPM','005930.KS','VZ','RELIANCE.NS','DLR','2010.SR',
'UPS','7203.T','PEP','MRK','1398.HK','MA','T']

In [0]:
datos = extrae.extraer_datos_yahoo(stocks)

[*********************100%***********************]  50 of 50 downloaded


Usando las fucniones auxiliares definidas previamente calculamos los rendimientos esperados y la matriz de varianza y covarianza para las acciones

In [0]:
mu = aux.calcular_rendimiento(datos)

In [0]:
S = aux.calcular_varianza(datos)

Calcularemos cuál fue el rendimiento máximo obtenido (de entre las medias de los datos históricos) y lo usaremos como el rendimiento deseado en el problema de optimización

In [0]:
r=max(mu).item()
r

0.4022108787760788

## Resolvemos con el Método de Newton usando diferencias finitas

La función a optimizar será la siguiente:

In [0]:
fo = lambda w: w@S@w

Usamos la solución analítica para tener la solución con la cual comparar.

In [0]:
w_ast = mkv.markowitz(r,mu,S)

La matriz $A$ de restricciones está dada por:

In [0]:
n = mu.shape[0]
A = cp.concatenate((mu,cp.ones(n))).reshape(2,n)
A

array([[ 0.13254293,  0.02782955,  0.25778345, -0.01787111, -0.02008539,
        -0.02286483,  0.19078421,  0.05743478,  0.17062275,  0.40221088,
         0.13618598,  0.02364206,  0.0476638 ,  0.13217291, -0.09666644,
        -0.03426582,  0.15880965,  0.17870289,  0.18557566,  0.15042008,
         0.08379133,  0.07154137,  0.08843336,  0.02202303,  0.07353272,
         0.13731909,  0.23605478,  0.19942876,  0.06806557,  0.26292508,
         0.07357148,  0.06803817,  0.03867589,  0.05090149,  0.14401416,
        -0.09339291,  0.23072551,  0.05036842,  0.05587177, -0.01264577,
        -0.01474117,  0.20678446,  0.06274014, -0.02729424,  0.1990038 ,
         0.07054765,  0.06563628,  0.04203765,  0.0717407 , -0.13227261],
       [ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.

Por otro lado el vector $b$ está dado por:

In [0]:
b = cp.array([r,1])
b

array([0.40221088, 1.        ])

Definimos el punto inicial con la función definida anteriormente:

In [0]:
w_0 = utils.feasible_markowitz(r,mu)

Definimos los parámetros para el método de Newton de la siguiente manera:

In [0]:
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=50

In [0]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)



I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0019	0.0009	1.1003	4.6964	---		11162.699
1	0.0019	0.0	0.0334	0.0002	1	11162.699
2	0.0019	0.0	0.0005	0.0	1	11162.699
Error of x with respect to x_ast: 0.00047875631346943525
Approximate solution: [ 1.58456068e-01 -2.08709832e-02  1.58048926e-01 -9.28997264e-02
  3.17031053e-02  5.10786741e-02  8.71853168e-02  1.26296186e-02
  3.72848119e-02  2.86001357e-01 -5.99024985e-03  2.24553601e-03
  2.03074487e-01  9.46087178e-02  2.28786290e-02  1.49784235e-02
  7.61265302e-03  2.96680495e-02  5.76467623e-02  1.98108419e-01
  1.19921167e-01  1.27865977e-01  1.41309880e-01  1.36173882e-02
  8.83887534e-02  1.50475380e-01  1.69271249e-01  7.72165387e-02
  8.09183143e-02  8.24853959e-02  1.92195287e-01 -2.39896300e-02
  2.64386780e-02  7.69729393e-02  2.20710514e-02 -1.08208343e-01
  1.64030583e-01  1.76988175e-02  6.16368407e-02 -1.08217542e-01
 -5.03608169e-02  1.38841035e-01  1.03558964e-01 -4.28149790e-02
  1.07055969e-02

La solución dada por el Método de Newton usando las diferenciación numérica es muy similar a la del método cerrado. Siendo el error de:

In [0]:
Err_plot[-1]

1.073648151492267e-08

In [0]:
w

array([ 1.58456068e-01, -2.08709832e-02,  1.58048926e-01, -9.28997264e-02,
        3.17031053e-02,  5.10786741e-02,  8.71853168e-02,  1.26296186e-02,
        3.72848119e-02,  2.86001357e-01, -5.99024985e-03,  2.24553601e-03,
        2.03074487e-01,  9.46087178e-02,  2.28786290e-02,  1.49784235e-02,
        7.61265302e-03,  2.96680495e-02,  5.76467623e-02,  1.98108419e-01,
        1.19921167e-01,  1.27865977e-01,  1.41309880e-01,  1.36173882e-02,
        8.83887534e-02,  1.50475380e-01,  1.69271249e-01,  7.72165387e-02,
        8.09183143e-02,  8.24853959e-02,  1.92195287e-01, -2.39896300e-02,
        2.64386780e-02,  7.69729393e-02,  2.20710514e-02, -1.08208343e-01,
        1.64030583e-01,  1.76988175e-02,  6.16368407e-02, -1.08217542e-01,
       -5.03608169e-02,  1.38841035e-01,  1.03558964e-01, -4.28149790e-02,
        1.07055969e-02, -2.33399846e+00,  1.90628556e-01,  2.63317339e-01,
        1.20089563e-01, -1.05514115e-01])

In [0]:
w_ast

array([ 1.58450459e-01, -2.08710279e-02,  1.58051613e-01, -9.28970852e-02,
        3.17093838e-02,  5.10824537e-02,  8.71800626e-02,  1.26389547e-02,
        3.72861469e-02,  2.86000507e-01, -5.98810147e-03,  2.24206232e-03,
        2.03075836e-01,  9.46030741e-02,  2.28766788e-02,  1.49919976e-02,
        7.60706433e-03,  2.96676402e-02,  5.76521006e-02,  1.98109863e-01,
        1.19928144e-01,  1.27869501e-01,  1.41300419e-01,  1.36285005e-02,
        8.83904753e-02,  1.50479914e-01,  1.69293512e-01,  7.72037012e-02,
        8.09121352e-02,  8.24658724e-02,  1.92197879e-01, -2.40095431e-02,
        2.64375219e-02,  7.69647088e-02,  2.20741648e-02, -1.08207562e-01,
        1.64032748e-01,  1.77020932e-02,  6.16398087e-02, -1.08210745e-01,
       -5.03462807e-02,  1.38834320e-01,  1.03567400e-01, -4.28198353e-02,
        1.06872745e-02, -2.33314798e+00,  1.89811544e-01,  2.63296457e-01,
        1.20089773e-01, -1.05535600e-01])

La varianza entre ambos métodos también es muy similar

In [0]:
w@S@w

array(9.45979256e-05)

In [0]:
w_ast@S@w_ast

array(9.45979246e-05)

In [0]:
utils.compute_error(w@S@w,w_ast@S@w_ast)

array(1.07364814e-08)

Además cumple con la restricción del rendimiento y la suma de pesos igual a 1

In [0]:
w@mu

array(0.40221088)

In [0]:
utils.compute_error(w@mu, r)

array(1.38015042e-16)

In [0]:
sum(w)

array(1.)

## Resolvemos con el Método de Newton usando funciones simbólicas

Ahora utilizaremos de nuevo el método de Newton usando las funciones simbólicas:

In [0]:
import inspect

In [0]:
lines = inspect.getsource(opt.gfo_cp_mark)
print(lines)

def gfo_cp_mark(Sigma,x):
    '''
    gradiente de la función objetivo 1/2*x.t*Sigma*x
    input: matriz Sigma y vector x
    output: producto matriz-vector Sigma*x
    '''
    first_block = Sigma@x
    return first_block



In [0]:
lines = inspect.getsource(opt.Hfo_cp_mark)
print(lines)

def Hfo_cp_mark(Sigma):
    '''
    Hessiana de la función objetivo 1/2*x.t*Sigma*x
    input: matriz Sigma
    output: matriz Sigma
    '''
    first_block = Sigma 
    return first_block



In [0]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter, opt.gfo_cp_mark, opt.Hfo_cp_mark,S)

TypeError: ignored

La solución dada por el Método de Newton usando las funciones simbólicas es prácticamente igual al método cerrado método cerrado. Siendo el error de:


In [0]:
Err_plot[-1]

In [0]:
w

In [0]:
w_ast

La varianza entre ambos métodos también es prácticamente la misma

In [0]:
w@S@w

In [0]:
w_ast@S@w_ast

In [0]:
utils.compute_error(w@S@w,w_ast@S@w_ast)

Y de igual manera cumple con la restricción del rendimiento y la suma de pesos igual a 1

In [0]:
w@mu

In [0]:
utils.compute_error(w@mu, r)

In [0]:
sum(w)

# Resultados




**1. Documentacion**

La Documentación de las funciones expresa de manera clara, consica y breve lo que hace cada una. De igual forma se explican de manera clara y concisa los argumentos de entrada y salida. La documentación es completa.


**2. Cumplimiento de objetivos del código**

Las funciones cumplen con sus objetivos.

**3. Test 1**

Revisaremos las función con distintas iteraciones y tolerancias.


## Método de Newton usando diferencias finitas

In [0]:
stocks = ['COP','AMT','LIN','LMT','AMZN','WMT','JNJ','VTI','MSFT','GOOG','XOM','CCI','BHP.AX','UNP',
'BABA','NSRGY','RHHBY','VOO','AAPL','FB','CVX','PLD','RIO.L','HON','HD','PG','UNH','BRK-A','V','0700.HK',
'RDSA.AS','0688.HK','AI.PA','RTX','MC.PA','KO','PFE','JPM','005930.KS','VZ','RELIANCE.NS','DLR','2010.SR',
'UPS','7203.T','PEP','MRK','1398.HK','MA','T']

datos = extrae.extraer_datos_yahoo(stocks)

[*********************100%***********************]  50 of 50 downloaded


In [0]:
mu = aux.calcular_rendimiento(datos)
S = aux.calcular_varianza(datos)
r=max(mu).item()
r

0.4022108787760788

In [0]:
fo = lambda w: w@S@w

In [0]:

w_ast = mkv.markowitz(r,mu,S)
n = mu.shape[0]
#Matriz A de restricciones
A = cp.concatenate((mu,cp.ones(n))).reshape(2,n)
#Vector b
b = cp.array([r,1])
#Punto inicial
w_0 = utils.feasible_markowitz(r,mu)

In [0]:
#Newton
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=50

In [0]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0019	0.0009	1.1003	4.6964	---		9106.4419
1	0.0019	0.0	0.0519	0.0002	1	9106.4419
2	0.0019	0.0	0.0011	0.0	1	9106.4419
Error of x with respect to x_ast: 0.001127150983856773
Approximate solution: [ 1.58457339e-01 -2.08741235e-02  1.58048326e-01 -9.29138166e-02
  3.17141968e-02  5.10597926e-02  8.71698378e-02  1.26222820e-02
  3.73103239e-02  2.86001815e-01 -5.98311667e-03  2.24920151e-03
  2.03043452e-01  9.45485132e-02  2.28809490e-02  1.50024690e-02
  7.61398944e-03  2.96568690e-02  5.76541033e-02  1.98006493e-01
  1.19877049e-01  1.27872221e-01  1.41306093e-01  1.35845928e-02
  8.83981380e-02  1.50482374e-01  1.69291553e-01  7.72193492e-02
  8.09252901e-02  8.24873544e-02  1.92209866e-01 -2.38880957e-02
  2.64351117e-02  7.69369561e-02  2.20693714e-02 -1.08235150e-01
  1.64055378e-01  1.76685729e-02  6.16358264e-02 -1.08221874e-01
 -5.02892406e-02  1.38835528e-01  1.03568021e-01 -4.28391427e-02
  1.07287887e-02 -

5.656699633119922e-08

In [0]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  5.656699313137432e-08
Error r:  1.380150417616186e-16
Error suma de pesos:  5.551115123125786e-16


In [0]:
#Newton
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500

In [0]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0019	0.0009	1.1003	4.6964	---		9106.4419
1	0.0019	0.0	0.0519	0.0002	1	9106.4419
2	0.0019	0.0	0.0011	0.0	1	9106.4419
Error of x with respect to x_ast: 0.001127150983856773
Approximate solution: [ 1.58457339e-01 -2.08741235e-02  1.58048326e-01 -9.29138166e-02
  3.17141968e-02  5.10597926e-02  8.71698378e-02  1.26222820e-02
  3.73103239e-02  2.86001815e-01 -5.98311667e-03  2.24920151e-03
  2.03043452e-01  9.45485132e-02  2.28809490e-02  1.50024690e-02
  7.61398944e-03  2.96568690e-02  5.76541033e-02  1.98006493e-01
  1.19877049e-01  1.27872221e-01  1.41306093e-01  1.35845928e-02
  8.83981380e-02  1.50482374e-01  1.69291553e-01  7.72193492e-02
  8.09252901e-02  8.24873544e-02  1.92209866e-01 -2.38880957e-02
  2.64351117e-02  7.69369561e-02  2.20693714e-02 -1.08235150e-01
  1.64055378e-01  1.76685729e-02  6.16358264e-02 -1.08221874e-01
 -5.02892406e-02  1.38835528e-01  1.03568021e-01 -4.28391427e-02
  1.07287887e-02 -

5.656699633119922e-08

In [0]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  5.656699313137432e-08
Error r:  1.380150417616186e-16
Error suma de pesos:  5.551115123125786e-16


In [0]:
#Newton
tol=1e-10
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500

In [0]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0019	0.0009	1.1003	4.6964	---		9106.4419
1	0.0019	0.0	0.0519	0.0002	1	9106.4419
2	0.0019	0.0	0.0011	0.0	1	9106.4419
Error of x with respect to x_ast: 0.001127150983856773
Approximate solution: [ 1.58457339e-01 -2.08741235e-02  1.58048326e-01 -9.29138166e-02
  3.17141968e-02  5.10597926e-02  8.71698378e-02  1.26222820e-02
  3.73103239e-02  2.86001815e-01 -5.98311667e-03  2.24920151e-03
  2.03043452e-01  9.45485132e-02  2.28809490e-02  1.50024690e-02
  7.61398944e-03  2.96568690e-02  5.76541033e-02  1.98006493e-01
  1.19877049e-01  1.27872221e-01  1.41306093e-01  1.35845928e-02
  8.83981380e-02  1.50482374e-01  1.69291553e-01  7.72193492e-02
  8.09252901e-02  8.24873544e-02  1.92209866e-01 -2.38880957e-02
  2.64351117e-02  7.69369561e-02  2.20693714e-02 -1.08235150e-01
  1.64055378e-01  1.76685729e-02  6.16358264e-02 -1.08221874e-01
 -5.02892406e-02  1.38835528e-01  1.03568021e-01 -4.28391427e-02
  1.07287887e-02 -

5.656699633119922e-08

In [0]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w))))). 

Error w:  5.656699313137432e-08
Error r:  1.380150417616186e-16
Error suma de pesos:  5.551115123125786e-16


In [0]:
#Newton
tol=1e-14
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500

In [0]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0019	0.0009	1.1003	4.6964	---		9106.4419
1	0.0019	0.0	0.0519	0.0002	1	9106.4419
2	0.0019	0.0	0.0011	0.0	1	9106.4419
3	0.0019	0.0	0.0	0.0	1	9106.4419
Error of x with respect to x_ast: 9.544841998625444e-06
Approximate solution: [ 1.58458855e-01 -2.08781432e-02  1.58050218e-01 -9.29027124e-02
  3.17230503e-02  5.10664183e-02  8.71685449e-02  1.26299781e-02
  3.73036140e-02  2.85996329e-01 -5.98471812e-03  2.24716268e-03
  2.03076331e-01  9.45639768e-02  2.28968193e-02  1.49818375e-02
  7.60512112e-03  2.96664083e-02  5.76561333e-02  1.98022653e-01
  1.19945869e-01  1.27860419e-01  1.41282215e-01  1.36057804e-02
  8.83763299e-02  1.50470160e-01  1.69300312e-01  7.72104560e-02
  8.09291837e-02  8.24697139e-02  1.92193230e-01 -2.39503930e-02
  2.64539815e-02  7.69287904e-02  2.20943127e-02 -1.08211855e-01
  1.64050712e-01  1.76898387e-02  6.16386869e-02 -1.08219827e-01
 -5.02639569e-02  1.38831445e-01  1.03577550e-01 

7.201681953971882e-12

In [0]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  7.201681953920019e-12
Error r:  2.7603008352323725e-16
Error suma de pesos:  2.2204460492503136e-16


In [0]:
#Newton
tol=1e-16
tol_backtracking=1e-15
p_ast=fo(w_ast)
maxiter=500

In [0]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0019	0.0009	1.1003	4.6964	---		9106.4419
1	0.0019	0.0	0.0519	0.0002	1	9106.4419
2	0.0019	0.0	0.0011	0.0	1	9106.4419
3	0.0019	0.0	0.0	0.0	1	9106.4419
4	0.0019	0.0	0.0	0.0	1	9106.4419
Error of x with respect to x_ast: 1.7551040903728132e-07
Approximate solution: [ 1.58458815e-01 -2.08781779e-02  1.58050235e-01 -9.29026415e-02
  3.17230574e-02  5.10667113e-02  8.71684745e-02  1.26297326e-02
  3.73032546e-02  2.85996602e-01 -5.98443652e-03  2.24694816e-03
  2.03077382e-01  9.45640142e-02  2.28968086e-02  1.49814625e-02
  7.60490281e-03  2.96663815e-02  5.76561287e-02  1.98023121e-01
  1.19945677e-01  1.27860429e-01  1.41282545e-01  1.36058259e-02
  8.83766151e-02  1.50469900e-01  1.69301552e-01  7.72103006e-02
  8.09296067e-02  8.24695180e-02  1.92193581e-01 -2.39500465e-02
  2.64542318e-02  7.69285250e-02  2.20948215e-02 -1.08211713e-01
  1.64050670e-01  1.76900936e-02  6.16387612e-02 -1.08219205e-01
 -5.02638359e-0

4.4124968505905025e-14

In [0]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  4.412496850590308e-14
Error r:  2.7603008352323725e-16
Error suma de pesos:  0.0


Ahora probamos con $r=0.3$

In [0]:
r=0.3
w_ast = mkv.markowitz(r,mu,S)
n = mu.shape[0]
#Matriz A de restricciones
A = cp.concatenate((mu,cp.ones(n))).reshape(2,n)
#Vector b
b = cp.array([r,1])
#Punto inicial
w_0 = utils.feasible_markowitz(r,mu)

In [0]:
#Newton
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=50


In [0]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]


I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0025	0.0021	1.3834	14.9538	---		10833.0647
1	0.0025	0.0	0.0952	0.0006	1	10833.0647
2	0.0025	0.0	0.0009	0.0	1	10833.0647
Error of x with respect to x_ast: 0.0009264514312908206
Approximate solution: [ 1.40214529e-01 -1.35978646e-02  1.10342203e-01 -4.37629688e-02
  4.34481638e-02  6.25194998e-02  6.23053196e-02  3.43246639e-02
  1.49933044e-02  2.18640943e-01 -1.88408397e-03  5.28946346e-04
  1.90524989e-01  8.92946732e-02  3.28308824e-03  1.04742689e-02
  6.47128709e-03  3.10463536e-02  5.03316129e-02  1.51109232e-01
  8.36292265e-02  1.09266126e-01  7.70736027e-02  3.71989934e-02
  7.11377668e-02  1.15367356e-01  1.14125980e-01  4.23057064e-02
  5.81986153e-02  4.24170188e-02  1.85545535e-01 -3.65085609e-02
  3.71677992e-02  7.24730267e-02 -3.16458803e-03 -8.39138211e-02
  1.36152098e-01  2.41561648e-02  4.86535869e-02 -8.05813009e-02
 -4.61480746e-02  9.42917043e-02  8.09904372e-02 -2.07954306e-02
  7.98583272e

3.6329669678523636e-08

In [0]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  3.6329668358678783e-08
Error r:  1.8503717077085946e-16
Error suma de pesos:  1.1102230246251554e-15


In [0]:
#Newton
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500

In [0]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0025	0.0021	1.3834	14.9538	---		10833.0647
1	0.0025	0.0	0.0952	0.0006	1	10833.0647
2	0.0025	0.0	0.0009	0.0	1	10833.0647
Error of x with respect to x_ast: 0.0009264514312908206
Approximate solution: [ 1.40214529e-01 -1.35978646e-02  1.10342203e-01 -4.37629688e-02
  4.34481638e-02  6.25194998e-02  6.23053196e-02  3.43246639e-02
  1.49933044e-02  2.18640943e-01 -1.88408397e-03  5.28946346e-04
  1.90524989e-01  8.92946732e-02  3.28308824e-03  1.04742689e-02
  6.47128709e-03  3.10463536e-02  5.03316129e-02  1.51109232e-01
  8.36292265e-02  1.09266126e-01  7.70736027e-02  3.71989934e-02
  7.11377668e-02  1.15367356e-01  1.14125980e-01  4.23057064e-02
  5.81986153e-02  4.24170188e-02  1.85545535e-01 -3.65085609e-02
  3.71677992e-02  7.24730267e-02 -3.16458803e-03 -8.39138211e-02
  1.36152098e-01  2.41561648e-02  4.86535869e-02 -8.05813009e-02
 -4.61480746e-02  9.42917043e-02  8.09904372e-02 -2.07954306e-02
  7.98583272e

3.6329669678523636e-08

In [0]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  3.6329668358678783e-08
Error r:  1.8503717077085946e-16
Error suma de pesos:  1.1102230246251554e-15


In [0]:
#Newton
tol=1e-10
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500

In [0]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0025	0.0021	1.3834	14.9538	---		10833.0647
1	0.0025	0.0	0.0952	0.0006	1	10833.0647
2	0.0025	0.0	0.0009	0.0	1	10833.0647
Error of x with respect to x_ast: 0.0009264514312908206
Approximate solution: [ 1.40214529e-01 -1.35978646e-02  1.10342203e-01 -4.37629688e-02
  4.34481638e-02  6.25194998e-02  6.23053196e-02  3.43246639e-02
  1.49933044e-02  2.18640943e-01 -1.88408397e-03  5.28946346e-04
  1.90524989e-01  8.92946732e-02  3.28308824e-03  1.04742689e-02
  6.47128709e-03  3.10463536e-02  5.03316129e-02  1.51109232e-01
  8.36292265e-02  1.09266126e-01  7.70736027e-02  3.71989934e-02
  7.11377668e-02  1.15367356e-01  1.14125980e-01  4.23057064e-02
  5.81986153e-02  4.24170188e-02  1.85545535e-01 -3.65085609e-02
  3.71677992e-02  7.24730267e-02 -3.16458803e-03 -8.39138211e-02
  1.36152098e-01  2.41561648e-02  4.86535869e-02 -8.05813009e-02
 -4.61480746e-02  9.42917043e-02  8.09904372e-02 -2.07954306e-02
  7.98583272e

3.6329669678523636e-08

In [0]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  3.6329668358678783e-08
Error r:  1.8503717077085946e-16
Error suma de pesos:  1.1102230246251554e-15


In [0]:
#Newton
tol=1e-14
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500

In [0]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0025	0.0021	1.3834	14.9538	---		10833.0647
1	0.0025	0.0	0.0952	0.0006	1	10833.0647
2	0.0025	0.0	0.0009	0.0	1	10833.0647
3	0.0025	0.0	0.0	0.0	1	10833.0647
Error of x with respect to x_ast: 1.3312483485567576e-06
Approximate solution: [ 1.40207439e-01 -1.35881195e-02  1.10340602e-01 -4.37791661e-02
  4.34450279e-02  6.25287481e-02  6.23429655e-02  3.43328374e-02
  1.50055433e-02  2.18632689e-01 -1.87368653e-03  5.28463589e-04
  1.90508063e-01  8.93042717e-02  3.27681854e-03  1.04912283e-02
  6.48286491e-03  3.10545150e-02  5.03437938e-02  1.51140538e-01
  8.36218275e-02  1.09292640e-01  7.70769980e-02  3.72109498e-02
  7.11397095e-02  1.15391539e-01  1.14129090e-01  4.23101917e-02
  5.82056832e-02  4.24236015e-02  1.85527975e-01 -3.64909426e-02
  3.71690877e-02  7.24600790e-02 -3.17248446e-03 -8.39242304e-02
  1.36147032e-01  2.41482752e-02  4.86551473e-02 -8.05947862e-02
 -4.61601086e-02  9.42946500e-02  8.1003682

1.5041258953619087e-12

In [0]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  1.5041258953596462e-12
Error r:  1.8503717077085946e-16
Error suma de pesos:  4.440892098500624e-16


In [0]:
#Newton
tol=1e-16
tol_backtracking=1e-15
p_ast=fo(w_ast)
maxiter=500

In [0]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0025	0.0021	1.3834	14.9538	---		10833.0647
1	0.0025	0.0	0.0952	0.0006	1	10833.0647
2	0.0025	0.0	0.0009	0.0	1	10833.0647
3	0.0025	0.0	0.0	0.0	1	10833.0647
Error of x with respect to x_ast: 1.3312483485567576e-06
Approximate solution: [ 1.40207439e-01 -1.35881195e-02  1.10340602e-01 -4.37791661e-02
  4.34450279e-02  6.25287481e-02  6.23429655e-02  3.43328374e-02
  1.50055433e-02  2.18632689e-01 -1.87368653e-03  5.28463589e-04
  1.90508063e-01  8.93042717e-02  3.27681854e-03  1.04912283e-02
  6.48286491e-03  3.10545150e-02  5.03437938e-02  1.51140538e-01
  8.36218275e-02  1.09292640e-01  7.70769980e-02  3.72109498e-02
  7.11397095e-02  1.15391539e-01  1.14129090e-01  4.23101917e-02
  5.82056832e-02  4.24236015e-02  1.85527975e-01 -3.64909426e-02
  3.71690877e-02  7.24600790e-02 -3.17248446e-03 -8.39242304e-02
  1.36147032e-01  2.41482752e-02  4.86551473e-02 -8.05947862e-02
 -4.61601086e-02  9.42946500e-02  8.1003682

1.5041258953619087e-12

In [0]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  1.5041258953596462e-12
Error r:  1.8503717077085946e-16
Error suma de pesos:  4.440892098500624e-16


Proseguimos a probar con $r=0.5$

In [0]:
r=0.5
w_ast = mkv.markowitz(r,mu,S)
n = mu.shape[0]
#Matriz A de restricciones
A = cp.concatenate((mu,cp.ones(n))).reshape(2,n)
#Vector b
b = cp.array([r,1])
#Punto inicial
w_0 = utils.feasible_markowitz(r,mu)


In [0]:
#Newton
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=50

In [0]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0079	0.0314	3.348	126.5304	---		4097.1375
1	0.0079	0.0002	2.982	0.6798	1	4097.1375
2	0.0079	0.0	0.2817	0.0046	1	4097.1375
3	0.0079	0.0	0.0097	0.0	1	4097.1375
Error of x with respect to x_ast: 0.009735388725341918
Approximate solution: [ 0.17592274 -0.02787265  0.20365926 -0.14000254  0.02052592  0.04005779
  0.11104618 -0.00835434  0.05850248  0.35042404 -0.00984593  0.00390479
  0.21512503  0.09982703  0.04165609  0.01941373  0.00864927  0.02834869
  0.06481175  0.24282919  0.15441967  0.14597598  0.20287954 -0.00906384
  0.1048823   0.18415892  0.22216095  0.1107827   0.10252299  0.12074375
  0.19847573 -0.01120149  0.01606237  0.08141158  0.04609985 -0.13146546
  0.19073341  0.01151288  0.07400231 -0.13472038 -0.05436273  0.18164784
  0.12507008 -0.06382252  0.01336091 -2.72781315  0.07383244  0.29235341
  0.12971612 -0.14898473]


3.880769742794788e-06

In [0]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  3.880754682479437e-06
Error r:  4.440892098500624e-16
Error suma de pesos:  1.5543122344752168e-15


In [0]:
#Newton
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500

In [0]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0079	0.0314	3.348	126.5304	---		4097.1375
1	0.0079	0.0002	2.982	0.6798	1	4097.1375
2	0.0079	0.0	0.2817	0.0046	1	4097.1375
3	0.0079	0.0	0.0097	0.0	1	4097.1375
Error of x with respect to x_ast: 0.009735388725341918
Approximate solution: [ 0.17592274 -0.02787265  0.20365926 -0.14000254  0.02052592  0.04005779
  0.11104618 -0.00835434  0.05850248  0.35042404 -0.00984593  0.00390479
  0.21512503  0.09982703  0.04165609  0.01941373  0.00864927  0.02834869
  0.06481175  0.24282919  0.15441967  0.14597598  0.20287954 -0.00906384
  0.1048823   0.18415892  0.22216095  0.1107827   0.10252299  0.12074375
  0.19847573 -0.01120149  0.01606237  0.08141158  0.04609985 -0.13146546
  0.19073341  0.01151288  0.07400231 -0.13472038 -0.05436273  0.18164784
  0.12507008 -0.06382252  0.01336091 -2.72781315  0.07383244  0.29235341
  0.12971612 -0.14898473]


3.880769742794788e-06

In [0]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  3.880754682479437e-06
Error r:  4.440892098500624e-16
Error suma de pesos:  1.5543122344752168e-15


In [0]:
#Newton
tol=1e-10
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500

In [0]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0079	0.0314	3.348	126.5304	---		4097.1375
1	0.0079	0.0002	2.982	0.6798	1	4097.1375
2	0.0079	0.0	0.2817	0.0046	1	4097.1375
3	0.0079	0.0	0.0097	0.0	1	4097.1375
4	0.0079	0.0	0.0002	0.0	1	4097.1375
Error of x with respect to x_ast: 0.00015119935653582983
Approximate solution: [ 0.17591928 -0.02785277  0.20370194 -0.13990762  0.02050847  0.04010047
  0.11091511 -0.00813046  0.0586331   0.35044576 -0.00991822  0.00389268
  0.21509838  0.09959806  0.04166895  0.01927614  0.0086735   0.02833636
  0.06464815  0.24287541  0.15470062  0.1456199   0.20270884 -0.0089714
  0.10486017  0.18402549  0.2220907   0.11059643  0.10266501  0.12077364
  0.19857703 -0.01196864  0.01620547  0.08120346  0.04627334 -0.13144992
  0.19074714  0.01151055  0.07406145 -0.13464727 -0.05419604  0.18143787
  0.12517371 -0.06390236  0.01334201 -2.70720578  0.05442419  0.29226417
  0.12957586 -0.14897831]


1.107005273044982e-09

In [0]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  1.1070052718195214e-09
Error r:  4.440892098500624e-16
Error suma de pesos:  8.881784197001244e-16


In [0]:
#Newton
tol=1e-14
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500

In [0]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0079	0.0314	3.348	126.5304	---		4097.1375
1	0.0079	0.0002	2.982	0.6798	1	4097.1375
2	0.0079	0.0	0.2817	0.0046	1	4097.1375
3	0.0079	0.0	0.0097	0.0	1	4097.1375
4	0.0079	0.0	0.0002	0.0	1	4097.1375
5	0.0079	0.0	0.0	0.0	1	4097.1375
Error of x with respect to x_ast: 6.728529011679431e-07
Approximate solution: [ 0.17592052 -0.02785289  0.20369578 -0.1399008   0.02050819  0.04010059
  0.11092015 -0.00813433  0.05863629  0.35044629 -0.00991737  0.00389108
  0.21510327  0.09959611  0.04166791  0.01927784  0.00867828  0.02833838
  0.06465227  0.24287752  0.15469797  0.14562463  0.20271041 -0.00897838
  0.10486785  0.18403086  0.22208703  0.11060041  0.10267043  0.12078294
  0.19857084 -0.01195184  0.01620285  0.0812043   0.04626941 -0.13144836
  0.19074722  0.01151135  0.07406065 -0.13464873 -0.05419024  0.18144179
  0.12517505 -0.06390294  0.01334568 -2.70754672  0.05469218  0.29226139
  0.12957546 -0.14896855]


2.0584956207058689e-13

In [0]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  2.058495620705445e-13
Error r:  4.440892098500624e-16
Error suma de pesos:  1.1102230246251554e-15


In [0]:
#Newton
tol=1e-16
tol_backtracking=1e-15
p_ast=fo(w_ast)
maxiter=500

In [0]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0079	0.0314	3.348	126.5304	---		4097.1375
1	0.0079	0.0002	2.982	0.6798	1	4097.1375
2	0.0079	0.0	0.2817	0.0046	1	4097.1375
3	0.0079	0.0	0.0097	0.0	1	4097.1375
4	0.0079	0.0	0.0002	0.0	1	4097.1375
5	0.0079	0.0	0.0	0.0	1	4097.1375
Error of x with respect to x_ast: 6.728529011679431e-07
Approximate solution: [ 0.17592052 -0.02785289  0.20369578 -0.1399008   0.02050819  0.04010059
  0.11092015 -0.00813433  0.05863629  0.35044629 -0.00991737  0.00389108
  0.21510327  0.09959611  0.04166791  0.01927784  0.00867828  0.02833838
  0.06465227  0.24287752  0.15469797  0.14562463  0.20271041 -0.00897838
  0.10486785  0.18403086  0.22208703  0.11060041  0.10267043  0.12078294
  0.19857084 -0.01195184  0.01620285  0.0812043   0.04626941 -0.13144836
  0.19074722  0.01151135  0.07406065 -0.13464873 -0.05419024  0.18144179
  0.12517505 -0.06390294  0.01334568 -2.70754672  0.05469218  0.29226139
  0.12957546 -0.14896855]


2.0584956207058689e-13

In [0]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  2.058495620705445e-13
Error r:  4.440892098500624e-16
Error suma de pesos:  1.1102230246251554e-15


## Método de Newton usando funciones simbólicas


# Hallazgos

El código funciona de manera correcta para distintos niveles de tolerancia, iteraciones y rendimientos r.

Las funciones generan unos valores con una exactitud de hasta 13 dígitos correctos. 



# Resultados 

In [0]:
r = 0.3
w_ast = mkv.markowitz(r,mu,S)
n = mu.shape[0]
#Matriz A de restricciones
A = cp.concatenate((mu,cp.ones(n))).reshape(2,n)
#Vector b
b = cp.array([r,1])
#Punto inicial
w_0 = utils.feasible_markowitz(r,mu)

In [0]:
#Newton
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=50


In [0]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0025	0.0021	1.3834	14.9538	---		10833.0647
1	0.0025	0.0	0.0952	0.0006	1	10833.0647
2	0.0025	0.0	0.0009	0.0	1	10833.0647
Error of x with respect to x_ast: 0.0009264514312908206
Approximate solution: [ 1.40214529e-01 -1.35978646e-02  1.10342203e-01 -4.37629688e-02
  4.34481638e-02  6.25194998e-02  6.23053196e-02  3.43246639e-02
  1.49933044e-02  2.18640943e-01 -1.88408397e-03  5.28946346e-04
  1.90524989e-01  8.92946732e-02  3.28308824e-03  1.04742689e-02
  6.47128709e-03  3.10463536e-02  5.03316129e-02  1.51109232e-01
  8.36292265e-02  1.09266126e-01  7.70736027e-02  3.71989934e-02
  7.11377668e-02  1.15367356e-01  1.14125980e-01  4.23057064e-02
  5.81986153e-02  4.24170188e-02  1.85545535e-01 -3.65085609e-02
  3.71677992e-02  7.24730267e-02 -3.16458803e-03 -8.39138211e-02
  1.36152098e-01  2.41561648e-02  4.86535869e-02 -8.05813009e-02
 -4.61480746e-02  9.42917043e-02  8.09904372e-02 -2.07954306e-02
  7.98583272e

3.6329669678523636e-08

In [0]:
#Newton
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500


In [0]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0025	0.0021	1.3834	14.9538	---		10833.0647
1	0.0025	0.0	0.0952	0.0006	1	10833.0647
2	0.0025	0.0	0.0009	0.0	1	10833.0647
Error of x with respect to x_ast: 0.0009264514312908206
Approximate solution: [ 1.40214529e-01 -1.35978646e-02  1.10342203e-01 -4.37629688e-02
  4.34481638e-02  6.25194998e-02  6.23053196e-02  3.43246639e-02
  1.49933044e-02  2.18640943e-01 -1.88408397e-03  5.28946346e-04
  1.90524989e-01  8.92946732e-02  3.28308824e-03  1.04742689e-02
  6.47128709e-03  3.10463536e-02  5.03316129e-02  1.51109232e-01
  8.36292265e-02  1.09266126e-01  7.70736027e-02  3.71989934e-02
  7.11377668e-02  1.15367356e-01  1.14125980e-01  4.23057064e-02
  5.81986153e-02  4.24170188e-02  1.85545535e-01 -3.65085609e-02
  3.71677992e-02  7.24730267e-02 -3.16458803e-03 -8.39138211e-02
  1.36152098e-01  2.41561648e-02  4.86535869e-02 -8.05813009e-02
 -4.61480746e-02  9.42917043e-02  8.09904372e-02 -2.07954306e-02
  7.98583272e

3.6329669678523636e-08

In [0]:
#Newton
tol=1e-14
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500

In [0]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0025	0.0021	1.3834	14.9538	---		10833.0647
1	0.0025	0.0	0.0952	0.0006	1	10833.0647
2	0.0025	0.0	0.0009	0.0	1	10833.0647
3	0.0025	0.0	0.0	0.0	1	10833.0647
Error of x with respect to x_ast: 1.3312483485567576e-06
Approximate solution: [ 1.40207439e-01 -1.35881195e-02  1.10340602e-01 -4.37791661e-02
  4.34450279e-02  6.25287481e-02  6.23429655e-02  3.43328374e-02
  1.50055433e-02  2.18632689e-01 -1.87368653e-03  5.28463589e-04
  1.90508063e-01  8.93042717e-02  3.27681854e-03  1.04912283e-02
  6.48286491e-03  3.10545150e-02  5.03437938e-02  1.51140538e-01
  8.36218275e-02  1.09292640e-01  7.70769980e-02  3.72109498e-02
  7.11397095e-02  1.15391539e-01  1.14129090e-01  4.23101917e-02
  5.82056832e-02  4.24236015e-02  1.85527975e-01 -3.64909426e-02
  3.71690877e-02  7.24600790e-02 -3.17248446e-03 -8.39242304e-02
  1.36147032e-01  2.41482752e-02  4.86551473e-02 -8.05947862e-02
 -4.61601086e-02  9.42946500e-02  8.1003682

1.5041258953619087e-12

In [0]:
#Newton
tol=1e-14
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=5000

In [0]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0025	0.0021	1.3834	14.9538	---		10833.0647
1	0.0025	0.0	0.0952	0.0006	1	10833.0647
2	0.0025	0.0	0.0009	0.0	1	10833.0647
3	0.0025	0.0	0.0	0.0	1	10833.0647
Error of x with respect to x_ast: 1.3312483485567576e-06
Approximate solution: [ 1.40207439e-01 -1.35881195e-02  1.10340602e-01 -4.37791661e-02
  4.34450279e-02  6.25287481e-02  6.23429655e-02  3.43328374e-02
  1.50055433e-02  2.18632689e-01 -1.87368653e-03  5.28463589e-04
  1.90508063e-01  8.93042717e-02  3.27681854e-03  1.04912283e-02
  6.48286491e-03  3.10545150e-02  5.03437938e-02  1.51140538e-01
  8.36218275e-02  1.09292640e-01  7.70769980e-02  3.72109498e-02
  7.11397095e-02  1.15391539e-01  1.14129090e-01  4.23101917e-02
  5.82056832e-02  4.24236015e-02  1.85527975e-01 -3.64909426e-02
  3.71690877e-02  7.24600790e-02 -3.17248446e-03 -8.39242304e-02
  1.36147032e-01  2.41482752e-02  4.86551473e-02 -8.05947862e-02
 -4.61601086e-02  9.42946500e-02  8.1003682

1.5041258953619087e-12

In [0]:
#Newton
tol=1e-16
tol_backtracking=1e-16
p_ast=fo(w_ast)
maxiter=5000

In [0]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0025	0.0021	1.3834	14.9538	---		10833.0647
1	0.0025	0.0	0.0952	0.0006	1	10833.0647
2	0.0025	0.0	0.0009	0.0	1	10833.0647
3	0.0025	0.0	0.0	0.0	1	10833.0647
Error of x with respect to x_ast: 1.3312483485567576e-06
Approximate solution: [ 1.40207439e-01 -1.35881195e-02  1.10340602e-01 -4.37791661e-02
  4.34450279e-02  6.25287481e-02  6.23429655e-02  3.43328374e-02
  1.50055433e-02  2.18632689e-01 -1.87368653e-03  5.28463589e-04
  1.90508063e-01  8.93042717e-02  3.27681854e-03  1.04912283e-02
  6.48286491e-03  3.10545150e-02  5.03437938e-02  1.51140538e-01
  8.36218275e-02  1.09292640e-01  7.70769980e-02  3.72109498e-02
  7.11397095e-02  1.15391539e-01  1.14129090e-01  4.23101917e-02
  5.82056832e-02  4.24236015e-02  1.85527975e-01 -3.64909426e-02
  3.71690877e-02  7.24600790e-02 -3.17248446e-03 -8.39242304e-02
  1.36147032e-01  2.41482752e-02  4.86551473e-02 -8.05947862e-02
 -4.61601086e-02  9.42946500e-02  8.1003682

1.5041258953619087e-12

In [0]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

error w:  1.5041258953596462e-12
error r;  1.8503717077085946e-16
error suma de pesos:  4.440892098500624e-16


r
